<a href="https://colab.research.google.com/github/IoT-gamer/sam2-dinov3-onnx/blob/main/notebooks/dinov3_onnx_export.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DINOv3 Feature Extractor ONNX Export

## Acknoweldgements/References
- [DINOv3 github repo](https://github.com/facebookresearch/dinov3)

## Dependencies

In [ ]:
!pip install onnx

In [ ]:
from google.colab import userdata
import torch
import onnx

## PyTorch Model Download
- visit https://ai.meta.com/resources/models-and-libraries/dinov3-downloads/ to get weights URLs
- add a secret named `dinov3_vits16` and copy and past the `dinov3_vits16` URL in the value
- note: the URLs may expire after a few days.
  - need to save or re-register

In [ ]:
MODEL_DINOV3_VITS = "dinov3_vits16"
MODEL_NAME = MODEL_DINOV3_VITS

WEIGHTS_URL = userdata.get('dinov3_vits16') # URL is stored in colab secrets

try:
  model = torch.hub.load(
      repo_or_dir="facebookresearch/dinov3",
      model=MODEL_NAME,
      source="github",
      weights=WEIGHTS_URL
  )
except Exception as e:
  print(e)
  print('may need to regisiter again at https://ai.meta.com/resources/models-and-libraries/dinov3-downloads/ ')

## Define PyTorch wrapper for DINOv3 Feature Extractor

In [ ]:
class DinoV3FeatureExtractor(torch.nn.Module):
    def __init__(self, model, n_layers):
        super().__init__()
        self.model = model
        self.n_layers = n_layers

    def forward(self, x):
        # Extract features from the last layer, normalized, as done during training
        features_list = self.model.get_intermediate_layers(x, n=range(self.n_layers), reshape=True, norm=True)
        last_layer_features = features_list[-1]  # Shape: (B, C, H_patches, W_patches)

        # Reshape for classifier: (B, C, H*W) -> (B, H*W, C)
        B, C, H, W = last_layer_features.shape
        features_reshaped = last_layer_features.view(B, C, -1)
        features_permuted = features_reshaped.permute(0, 2, 1)
        return features_permuted

## Export DINOv3 Feature Extractor to ONNX

In [ ]:
image_size = 768 # for dummy input (image size can be variable)
n_layers = 12 # for dinov3_vits16
onnx_feature_extractor_path = "dinov3_feature_extractor.onnx"
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = "cpu"
onnx_exportable_dino = DinoV3FeatureExtractor(model, n_layers).to(device).eval()

# Dummy input with dynamic axes for variable image sizes
dummy_input = torch.randn(1, 3, image_size, image_size, device=device)

print(f"Exporting DINOv3 feature extractor to {onnx_feature_extractor_path}...")
torch.onnx.export(
    onnx_exportable_dino,
    dummy_input,
    onnx_feature_extractor_path,
    input_names=['input_image'],
    output_names=['patch_features'],
    dynamic_axes={
        'input_image': {2: 'height', 3: 'width'},
        'patch_features': {1: 'num_patches'}
    },
    opset_version=17
)
print("DINOv3 ONNX export complete!")